# Parallel Function Calling

Parallel function calls are available in the following models: gpt-4-1106-preview and gpt-3.5-turbo-1106.

For the most accurate and detailed information, please visit the [OpenAI documentation directly at OpenAI Platform Guides - Function Calling](https://platform.openai.com/docs/guides/function-calling/parallel-function-calling).

In [1]:
import logging
import os
import openai
import itertools
from openai import OpenAI
from actionweaver.llms import patch


openai.api_key = os.getenv("OPENAI_API_KEY")

openai_client = patch(OpenAI())

In [2]:
from actionweaver import action

@action(name="GetWeather")
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    print ("Getting current weather")
    
    import json
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


@action(name="GetCurrentTime")
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    print ("Getting current time")
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"


In [3]:
response = openai_client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[{"role": "user", "content": "what time is it now and What's the weather like in San Francisco, Tokyo, and Paris?"}],
  actions = [get_current_weather, get_current_time],
  stream=False, 
)


Getting current weather
Getting current weather
Getting current weather
Getting current time


In [4]:
print (response.choices[0].message.content)

The current time is 09:05 AM, and the weather is as follows:
- San Francisco: 72°F
- Tokyo: 10°C
- Paris: 22°C
